In [3]:
pip install -r ./requirements.txt

  Using cached openai-1.12.0-py3-none-any.whl.metadata (18 kB)
  Using cached motor-3.4.0-py3-none-any.whl.metadata (21 kB)
  Using cached prance-23.6.21.0-py3-none-any.whl.metadata (13 kB)
  Using cached regex-2023.12.25-cp311-cp311-macosx_10_9_x86_64.whl.metadata (40 kB)
  Using cached anyio-4.3.0-py3-none-any.whl.metadata (4.6 kB)
  Using cached distro-1.9.0-py3-none-any.whl.metadata (6.8 kB)
  Using cached httpx-0.27.0-py3-none-any.whl.metadata (7.2 kB)
  Using cached sniffio-1.3.1-py3-none-any.whl.metadata (3.9 kB)
  Using cached httpcore-1.0.5-py3-none-any.whl.metadata (20 kB)
  Using cached h11-0.14.0-py3-none-any.whl.metadata (8.2 kB)
  Using cached pymongo-4.7.0-cp311-cp311-macosx_10_9_x86_64.whl.metadata (22 kB)
  Using cached asgiref-3.8.1-py3-none-any.whl.metadata (9.3 kB)
  Using cached more_itertools-10.2.0-py3-none-any.whl.metadata (34 kB)
  Using cached openapi_schema_validator-0.6.2-py3-none-any.whl.metadata (5.3 kB)
  Using cached openapi_spec_validator-0.7.1-py3-none

In [1]:
import semantic_kernel as sk
import asyncio
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.planning import SequentialPlanner
from dotenv import load_dotenv
import os
from plugins.QueryDb import queryDb as plugin
from semantic_kernel.planning import StepwisePlanner

# Get ROOT_DIR

# Use '__file__' only if it's defined
if '__file__' in globals():
    ROOT_DIR = os.path.dirname(os.path.dirname(os.path.abspath(__file__)))
else:
    # If '__file__' is not defined, set a default ROOT_DIR
    ROOT_DIR = os.path.abspath(os.getcwd())

# Take environment variables from .env.
load_dotenv(os.path.join(ROOT_DIR, '.env'), override=True)

# Take environment variables from .env.
load_dotenv(ROOT_DIR+'/.env', override=True)

async def create_plan(planner, input):
    return await planner.create_plan(goal=input)

async def invoke_plan(sequential_plan):
    return await sequential_plan.invoke()

async def main(nlp_input):
    
    kernel = sk.Kernel()
    
    # Get AOAI settings from .env
    deployment, api_key, endpoint = sk.azure_openai_settings_from_dot_env()

    # Set the deployment name to the value of your chat model
    deployment, api_key, endpoint = sk.azure_openai_settings_from_dot_env()
    azure_text_service = AzureChatCompletion(deployment_name=deployment, endpoint=endpoint, api_key=api_key)
    kernel.add_text_completion_service("azure_text_completion", azure_text_service)

    # Immport NLP to SQL Plugin
    plugins_directory = "plugins"
    kernel.import_semantic_plugin_from_directory(plugins_directory, "nlpToSqlPlugin")
    kernel.import_plugin(plugin.QueryDbPlugin(os.getenv("CONNECTION_STRING")), plugin_name="QueryDbPlugin")
    
    planner = SequentialPlanner(kernel)
    #planner = StepwisePlanner(kernel)

    # Create a plan with the NLP input
    ask = f"Create a SQL query according to the following request: {nlp_input} and query the database to get the result."

    plan = await create_plan(planner, ask)
 
    # Invoke the plan and get the result
    result = await invoke_plan(plan)

    print('/n')
    print(f'User ASK: {nlp_input}')
    print(f'Response: {result}')
    print('/n')
    
    # Print each step of the plan and its result
    for index, step in enumerate(plan._steps):
        print("Step:", index)
        print("Description:", step.description)
        print("Function:", step.plugin_name + "." + step._function.name)
        if len(step._outputs) > 0:
            print("  Output:\n", str.replace(result[step._outputs[0]], "\n", "\n  "))
            print("\n\n")

In [2]:
result = await main("I want to know how many transactions in the last 3 months")
print(result)

/n
User ASK: I want to know how many transactions in the last 3 months
Response: Response: According to the database query, the number of transactions is 22.
/n
Step: 0
Description: Write SQL queries given a Natural Language description
Function: nlpToSqlPlugin.ConvertNLPToSQL
  Output:
 SELECT COUNT(*) AS NumberOfTransactions
  FROM sales_transaction
  WHERE transaction_date >= DATEADD(MONTH, -3, GETDATE());



Step: 1
Description: Convert SQL queries in any ANSI SQL dialect to a Transact-SQL dialect
Function: nlpToSqlPlugin.MakeSQLCompatible
  Output:
 SELECT COUNT(*) AS NumberOfTransactions
  FROM sales_transaction
  WHERE transaction_date >= DATEADD(MONTH, -3, GETDATE());



Step: 2
Description: Query a database using a SQL query
Function: QueryDbPlugin.query_db
  Output:
 [{'NumberOfTransactions': 22}]



Step: 3
Description: Write a friendly response given a database query result
Function: nlpToSqlPlugin.WriteResponse
  Output:
 Response: According to the database query, the numb

In [16]:
result = await main("Give me the name of the best seller in terms of sales volume in the whole period")
print(result)

/n
User ASK: Give me the name of the best seller in terms of sales volume in the whole period
Response: Response: [{'seller_name': 'Bob Johnson'}]
Message -> The seller's name according to the database query is Bob Johnson.
/n
Step: 0
Description: Write SQL queries given a Natural Language description
Function: nlpToSqlPlugin.ConvertNLPToSQL
  Output:
 SELECT TOP 1 s.seller_name
  FROM sellers s
  JOIN sales_transaction st ON s.seller_id = st.seller_id
  GROUP BY s.seller_name
  ORDER BY SUM(st.quantity) DESC;



Step: 1
Description: Convert SQL queries in any ANSI SQL dialect to a Transact-SQL dialect
Function: nlpToSqlPlugin.MakeSQLCompatible
  Output:
 SELECT TOP 1 s.seller_name
  FROM sellers s
  JOIN sales_transaction st ON s.seller_id = st.seller_id
  GROUP BY s.seller_name
  ORDER BY SUM(st.quantity) DESC;



Step: 2
Description: Query a database using a SQL query
Function: QueryDbPlugin.query_db
  Output:
 [{'seller_name': 'Bob Johnson'}]



Step: 3
Description: Write a friendl